In [1]:
# Utils
'''
Plot loss and metrics of keras training.
'''
def utils_plot_keras_training(training):
    metrics = [k for k in training.history.keys() if ("loss" not in k) and ("val" not in k)]
    fig, ax = plt.subplots(nrows=1, ncols=2, sharey=True, figsize=(15,3))
    
    ## training
    ax[0].set(title="Training")
    ax11 = ax[0].twinx()
    ax[0].plot(training.history['loss'], color='black')
    ax[0].set_xlabel('Epochs')
    ax[0].set_ylabel('Loss', color='black')
    for metric in metrics:
        ax11.plot(training.history[metric], label=metric)
    ax11.set_ylabel("Score", color='steelblue')
    ax11.legend()
    
    ## validation
    ax[1].set(title="Validation")
    ax22 = ax[1].twinx()
    ax[1].plot(training.history['val_loss'], color='black')
    ax[1].set_xlabel('Epochs')
    ax[1].set_ylabel('Loss', color='black')
    for metric in metrics:
        ax22.plot(training.history['val_'+metric], label=metric)
    ax22.set_ylabel("Score", color="steelblue")
    plt.show()

In [2]:
from sklearn.metrics import r2_score

def test(ytrue, ypred, ydiff, cutoff):
    r2_model=r2_score(ytrue, ypred) # Calculate r2_score of predictions

    squarederror_model = (ytrue-ypred)**2 # Calculate SE of predictions
    MSE_model=squarederror_model.mean() # MSE

    abserror_model = np.abs(ytrue-ypred) # Calculate AE of predictions
    MAE_model = abserror_model.mean()

    APE_model = 100*np.abs((ytrue-ypred)/ytrue) # Calculate APE of predictions
    MAPE_model = APE_model.mean()
    positives_obs = (ytrue>=cutoff).sum()
    negatives_obs = (ytrue<cutoff).sum()
    positives_pred = (ypred>=cutoff).sum()
    negatives_pred = (ypred<cutoff).sum()

    tn = np.sum((ytrue<cutoff)&(ypred<cutoff))
    tp = np.sum((ytrue>=cutoff)&(ypred>=cutoff))
    fn = np.sum((ytrue>=cutoff)&(ypred<cutoff))
    fp = np.sum((ytrue<cutoff)&(ypred>=cutoff))

    #perc=test[test['overall']>=4].shape[0]/test.shape[0]
    #q=test['yhat'].quantile(1-perc)
    #positives_c = test[test['yhat']>=q].shape[0]
    #negatives_c = test[test['yhat']<q].shape[0]

    accuracy = (tp+tn)/(tp+tn+fp+fn)
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    f1=2*(precision*recall)/(precision+recall)
    specificity = tn/(tn+fp)
    
    # Hvor mange er forudsagt mindre end de faktisk er
    under_modeln=np.sum(ydiff<0)

    # Hvor mange er forudsagt større end de faktisk er
    over_modeln=np.sum(ydiff>0)
    
    # Lige på
    equal_modeln = np.sum(ydiff==0)


    KPI_model = np.array([r2_model, MSE_model, MAE_model, MAPE_model, under_modeln, over_modeln, equal_modeln,
                         tp, tn, fp, fn, accuracy, precision, recall, f1, specificity])
    KPIdf=pd.DataFrame(KPI_model).T
    KPIdf.columns=['r2', 'MSE', 'MAE', 'MAPE', 'pred<true', 'pred>true', 'equal', 'TP', 'TN', 'FP', 'FN',
                  'Accuracy', 'Precision', 'Recall', 'f1', 'Specificity']


    return KPIdf, KPI_model